In [1]:
import re
import matplotlib.pyplot as plt
from matplotlib.collections import EventCollection
import numpy as np

In [2]:
class pathdata(object):
    def __init__(self, sourcefile):
        self.source = sourcefile
        self.epochs = []
        self.dist = []
        self.sparsity = []
        self.finalerr = 0
        self.finalspar = 0
        self.percav = {}
        
        self.fill_data()
        
        
    def fill_data(self):
        with open(self.source, "r") as myfile:
            file_learned = True
            for line in myfile:
                is_train = re.search("Epoch:", line)
                val_los = re.search("Loss: \d+\.\d+", line)
                percentile = re.search("Percentile \d+:", line)
                if is_train:
                    self.epochs.append(int(re.search("\d+", line).group()))
                    self.dist.append(float(re.search("\d+\.\d+", val_los.group()).group()))
                    self.sparsity.append(float(re.search("\d+\.\d+", re.search("Sparsity: \d+\.\d+", line).group()).group()))
                elif re.search("Test Loss:", line):
                    self.finalerr = float(re.search("\d+\.\d+", val_los.group()).group())
                    self.finalspar = float(re.search("\d+\.\d+", re.search("Sparsity: \d+\.\d+", line).group()).group())
                elif percentile:
                    p = int(re.search("\d+", percentile.group()).group())
                    self.percav[p] = float(re.search("\d+\.\d+", val_los.group()).group())
                if file_learned and re.search("nan", line):
                    file_learned = False
                    print("Error occurred in file {}".format(self.source))

In [3]:
class summarydata_withstd(object):
    def __init__(self, files):
        self.files = files
        self.avgerror = 0
        self.avgerror_std = None
        self.avgsparsity = 0
        self.avgsparsity_std = None
        
        self.percs = {}
        self.percs_std = {}
        
        self.summary()
        
    def summary(self):
        pathdatadict = {}
        percs1 = {20: [], 40 : [], 60: [], 80: [], 90: [], 95: []}
        for i in range(len(self.files)):
            pathdatadict[i] = pathdata(self.files[i])
            for p in percs1.keys():
                percs1[p].append(pathdatadict[i].percav[p])
        err = [pathdatadict[i].finalerr for i in range(len(self.files))]
        sparse = [pathdatadict[i].finalspar for i in range(len(self.files))]
            
        self.avgerror = np.mean(err)
        self.avgerror_std = np.std(err)
        self.avgsparsity = np.mean(sparse)
        self.avgsparsity_std = np.std(sparse)
        for p in percs1.keys():
            self.percs[p] = np.nanmean(percs1[p])
            self.percs_std[p] = np.nanstd(percs1[p])
              

In [4]:
class tablemaker_withstd(object):
    def __init__(self, filedict):
        self.filedict = filedict
        self.filenames = []
        self.bes = []
        self.be_std = []
        self.ps = {20: [], 40 : [], 60: [], 80: [], 90: [], 95: []}
        self.ps_std = {20: [], 40 : [], 60: [], 80: [], 90: [], 95: []}
        
        self.makedata()
        
    def makedata(self):
        for f in self.filedict.keys():
            self.filenames.append(f)
            fsum = summarydata_withstd(self.filedict[f])
            self.bes.append(fsum.avgerror)
            self.be_std.append(fsum.avgerror_std)
            for p in self.ps.keys():
                self.ps[p].append(fsum.percs[p])
                self.ps_std[p].append(fsum.percs_std[p])
                
    def maketable(self):
        print(f'\\begin{{tabular}}{{c|' + ''.join(['c' for i in range(len(self.filenames))]) + f'}} \\hline')
        print(' & ' + ' & '.join(self.filenames) + f' \\\\ \\hline')
        print('{{\\bf Test loss:}} & ' + ' & '.join(['{:.2f} ({:.2f})'.format(self.bes[i], self.be_std[i]) for i in range(len(self.bes))]) + f' \\\\')
        for p in self.ps.keys():
            print('{{\\bf {}\\% Sparse Err.}} & '.format(str(p)) + ' & '.join(['{:.2f} ({:.2f})'.format(self.ps[p][i], self.ps_std[p][i]) for i in range(len(self.ps[p]))]) + f' \\\\')
        print(f'\\end{{tabular}}')

In [7]:
mylist = [1, 2, 3, 4]
modulidict = {}
modulidict['Torus'] = ['traindata/torusl1dog{}.txt'.format(k) for k in mylist]
modulidict['Circle'] = ['traindata/circlel1dog{}.txt'.format(k) for k in mylist]
modulidict['Sphere'] = ['traindata/spherel1dog{}.txt'.format(k) for k in mylist]
modulidict['Klein'] = ['traindata/kleinl1dog{}.txt'.format(k) for k in mylist]
modulidict['6-Torus'] = ['traindata/torus6l1dog{}.txt'.format(k) for k in mylist]
moduli = tablemaker_withstd(modulidict)

controldict = {}
controldict['Torus'] = ['traindata/torusl1dog{}.txt'.format(k) for k in mylist]
controldict['No reg.'] = ['traindata/noreg{}.txt'.format(k) for k in mylist]
controldict['$L_1$'] = ['traindata/l1{}.txt'.format(k) for k in mylist]
#controldict['$L_1^*$'] = ['traindata/meanl1{}.txt'.format(k) for k in mylist]
controldict['Perm. DoG'] = ['traindata/perml1dog{}.txt'.format(k) for k in mylist]
controldict['Perm. Diff.'] = ['traindata/perml1sq{}.txt'.format(k) for k in mylist]
control = tablemaker_withstd(controldict)

KeyError: 20